In [155]:
# Libraries
import pandas as pd
import numpy as np
from pandas_profiling import ProfileReport 
import missingno as msno
import matplotlib.pyplot as plt
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_squared_error
from category_encoders import TargetEncoder
from sklearn.model_selection import train_test_split

In [156]:
# import 

store = pd.read_csv("./data/store.csv")
sales = pd.read_csv("./data/train.csv")

/home/ivelina_zaharieva/anaconda3/envs/competition_env/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [157]:
sales = sales.dropna()

In [158]:
store.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


In [159]:
sales.head()

,Date,Store,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,2013-01-01,1115.0,2.0,0.0,0.0,0.0,0.0,a,1.0
1,2013-01-01,379.0,2.0,0.0,0.0,0.0,0.0,a,1.0
2,2013-01-01,378.0,2.0,0.0,0.0,0.0,0.0,a,1.0
3,2013-01-01,377.0,2.0,0.0,0.0,0.0,0.0,a,1.0
4,2013-01-01,376.0,2.0,0.0,0.0,0.0,0.0,a,1.0


In [160]:
# merge

rossman_df = pd.merge(sales, store, how='left', on='Store')


In [161]:
X = rossman_df.drop(columns="Sales")
y = rossman_df.loc[:, "Sales"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

In [162]:
train_dataset = pd.concat([X_train, y_train], axis=1)
train_dataset.head()

,Date,Store,DayOfWeek,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,Sales
301442,2013-12-12,947.0,4.0,1170.0,1.0,0.0,0,0.0,a,a,460.0,3.0,2014.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct",8426.0
83900,2013-04-07,427.0,7.0,0.0,0.0,0.0,0,0.0,a,c,70.0,7.0,2005.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct",0.0
49859,2013-02-26,762.0,2.0,290.0,1.0,0.0,0,0.0,d,c,1280.0,NaN,NaN,1,10.0,2013.0,"Mar,Jun,Sept,Dec",2905.0
46579,2013-02-23,85.0,6.0,789.0,1.0,0.0,0,0.0,b,a,1870.0,10.0,2011.0,0,NaN,NaN,NaN,5349.0
46311,2013-02-22,703.0,5.0,392.0,1.0,1.0,0,0.0,a,a,80.0,6.0,2005.0,0,NaN,NaN,NaN,3606.0


In [163]:
# sanity check
train_dataset = train_dataset[~((train_dataset.Sales<1)&(train_dataset.Open==1))]

In [ ]:
# drop closed days
# drop "Open" column

train_dataset = train_dataset[~(train_dataset["Open"]==0)]
train_dataset = train_dataset.drop(["Open"], axis=1)
len(train_dataset)

In [164]:
# convert cols to categorical
to_cat = ["Store", "DayOfWeek", "Open", "Promo", "SchoolHoliday", "StateHoliday"]
for i in to_cat:
    train_dataset.loc[:, i] = pd.Categorical(train_dataset[i])

In [166]:
# encoding date

def add_time_features(df):
    df['Year'] = df.Date.dt.year
    df['Month'] = df.Date.dt.month
    df['Day'] = df.Date.dt.day
    df['DayOfWeek'] = df.Date.dt.dayofweek
    df['WeekOfYear'] = df.Date.dt.weekofyear
    return df

train_dataset['Date'] = pd.to_datetime(train_dataset["Date"])
train_dataset = add_time_features(train_dataset)

/home/ivelina_zaharieva/anaconda3/envs/competition_env/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  


In [167]:
assortment_encoder = TargetEncoder()
train_dataset.loc[:, "Assortment"] = assortment_encoder.fit_transform(
                                        train_dataset.loc[:, "Assortment"],
                                        train_dataset.loc[:, "Sales"]) 

In [168]:
train_dataset.loc[:, "Assortment"]

301442    6572.721113
49859     7122.291328
46579     6572.721113
46311     6572.721113
126293    6572.721113
             ...     
119879    7122.291328
259178    6572.721113
365838    7122.291328
131932    6572.721113
146867    7122.291328
Name: Assortment, Length: 351716, dtype: float64

In [169]:
train_dataset.loc[:, "Customers"].nunique()

3682

In [170]:
train_dataset.loc[:, "Store"].nunique()

1115

In [171]:
store_customers = train_dataset.groupby("Store").mean()["Customers"]
store_customers

Store
1.0        574.000000
2.0        586.952532
3.0        744.506329
4.0       1316.430341
5.0        540.794953
             ...     
1111.0     472.452012
1112.0     893.401869
1113.0     724.612426
1114.0    3191.695238
1115.0     409.080745
Name: Customers, Length: 1115, dtype: float64

In [172]:
train_dataset = pd.merge(train_dataset, store_customers, how='left', on='Store')
len(train_dataset)

351716

In [173]:
train_dataset.loc[train_dataset.loc[:, "Store"]==1]

,Date,Store,DayOfWeek,Customers_x,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,...,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,Sales,Year,Month,Day,WeekOfYear,Customers_y
1425,2013-10-11,1.0,4,670.0,1.0,0,0.0,c,6572.721113,1270.0,...,0,NaN,NaN,NaN,5896.0,2013,10,11,41,574.0
2802,2013-02-01,1.0,4,658.0,0.0,0,0.0,c,6572.721113,1270.0,...,0,NaN,NaN,NaN,5633.0,2013,2,1,5,574.0
4142,2013-02-19,1.0,1,608.0,1.0,0,0.0,c,6572.721113,1270.0,...,0,NaN,NaN,NaN,5386.0,2013,2,19,8,574.0
4696,2014-03-29,1.0,5,548.0,0.0,0.0,0.0,c,6572.721113,1270.0,...,0,NaN,NaN,NaN,4583.0,2014,3,29,13,574.0
5947,2014-01-17,1.0,4,479.0,0.0,0,0.0,c,6572.721113,1270.0,...,0,NaN,NaN,NaN,3803.0,2014,1,17,3,574.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
344479,2013-08-12,1.0,0,600.0,1.0,0,1.0,c,6572.721113,1270.0,...,0,NaN,NaN,NaN,5326.0,2013,8,12,33,574.0
346474,2014-05-07,1.0,2,604.0,1.0,0,0.0,c,6572.721113,1270.0,...,0,NaN,NaN,NaN,5075.0,2014,5,7,19,574.0
346994,2014-04-16,1.0,2,662.0,1.0,0,1.0,c,6572.721113,1270.0,...,0,NaN,NaN,NaN,6381.0,2014,4,16,16,574.0
349354,2013-06-10,1.0,0,531.0,0.0,0,0.0,c,6572.721113,1270.0,...,0,NaN,NaN,NaN,4291.0,2013,6,10,24,574.0


In [175]:
store_day_customers = pd.DataFrame(train_dataset.groupby(["Store", "DayOfWeek"]).mean()["Customers_x"]).reset_index()
store_day_customers

,Store,DayOfWeek,Customers_x
0,1.0,0,616.035714
1,1.0,1,546.381818
2,1.0,2,553.826923
3,1.0,3,549.875000
4,1.0,4,579.631579
...,...,...,...
7800,1115.0,2,381.090909
7801,1115.0,3,373.291667
7802,1115.0,4,432.063830
7803,1115.0,5,465.945455


In [149]:
train_dataset = pd.merge(train_dataset, store_day_customers, how='left', on=('Store', 'DayOfWeek'))
len(train_dataset)

351716

In [151]:
train_dataset.loc[train_dataset.loc[:, "Store"]==1].head()

,Date,Store,DayOfWeek,Customers_x_x,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,...,Promo2SinceWeek,Promo2SinceYear,PromoInterval,Sales,Year,Month,Day,WeekOfYear,Customers_y,Customers_x_y
1425,2013-10-11,1.0,4,670.0,1.0,0,0.0,c,6572.721113,1270.0,...,NaN,NaN,NaN,5896.0,2013,10,11,41,574.0,579.631579
2802,2013-02-01,1.0,4,658.0,0.0,0,0.0,c,6572.721113,1270.0,...,NaN,NaN,NaN,5633.0,2013,2,1,5,574.0,579.631579
4142,2013-02-19,1.0,1,608.0,1.0,0,0.0,c,6572.721113,1270.0,...,NaN,NaN,NaN,5386.0,2013,2,19,8,574.0,546.381818
4696,2014-03-29,1.0,5,548.0,0.0,0.0,0.0,c,6572.721113,1270.0,...,NaN,NaN,NaN,4583.0,2014,3,29,13,574.0,601.043478
5947,2014-01-17,1.0,4,479.0,0.0,0,0.0,c,6572.721113,1270.0,...,NaN,NaN,NaN,3803.0,2014,1,17,3,574.0,579.631579
